#Transformer-Based Sentiment Analysis

##Goal of the project:

Learn how NLP/AI can be used to solve real-world problems. We have the data for 49 million tweets, pre-dominantly focused on the Pakistani Twitter sphere. We want to use this data to figure a few things out.

We want to figure out
Twitter accounts that are biased in favour/against of a specific party (i.e. tweets in favour of a specific political party)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv("/content/drive/MyDrive/sample.csv")
df_raw = pd.read_csv("/content/drive/MyDrive/sample.csv")

**EDA (EXPLORATORY DATA ANALYSIS)**

In [4]:
df

,index,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,display_text_range,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,withheld_in_countries,extended_tweet,created_day
0,0,2022-04-26 10:37:27+00:00,1518902068735533056,1518902068735533056,RT @fawadchaudhry: آج الیکشن کمیشن کے دفاتر کے...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
1,1,2022-04-26 10:37:27+00:00,1518902068743913472,1518902068743913472,RT @ImranRiazKhan: عمران خان کی تقریر کا پاکست...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
2,2,2022-04-26 10:37:27+00:00,1518902068878192640,1518902068878192640,RT @InsafPK: پاکستان تحریک انصاف پنجاب کا صوبا...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
3,3,2022-04-26 10:37:27+00:00,1518902068953681922,1518902068953681920,RT @ZBasharit: پاکستان رمضان کے مہینے میں آزاد...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
4,4,2022-04-26 10:37:27+00:00,1518902069050380290,1518902069050380288,RT @ImranRiazKhan: مبینہ طور پر ٹویٹر کی تاریخ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2022-04-26 10:37:47+00:00,1518902151338020864,1518902151338020864,RT @SadiaAmber10: #امپورٹڈ_حکومت_نامنظور since...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,...,NaN,{'media': array([{'additional_media_info': Non...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
996,996,2022-04-26 10:37:47+00:00,1518902151191351297,1518902151191351296,نیوٹرل بھڑوے باجوے نے مدد کی چوروں کی اور اپنی...,"<a href=""http://twitter.com/download/iphone"" r...",True,NaN,NaN,NaN,...,NaN,NaN,NaN,1.518812e+18,1.518812e+18,"{'contributors': None, 'coordinates': None, 'c...","{'display': 'twitter.com/ambreenpti1/st…', 'ex...",NaN,"{'display_text_range': array([ 0, 137]), 'ent...",2022-04-26
997,997,2022-04-26 10:37:47+00:00,1518902151510208514,1518902151510208512,@SSEHBAI1 محترم سچی بات کریں گے تو متوالے ساتھ...,"<a href=""http://twitter.com/download/iphone"" r...",False,1.518745e+18,1.518745e+18,234092394.0,...,[10 78],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26
998,998,2022-04-26 10:37:47+00:00,1518902151128489986,1518902151128489984,Those hands should be in handcuffs #امپورٹڈ_حک...,"<a href=""http://twitter.com/#!/download/ipad"" ...",False,NaN,NaN,NaN,...,[ 0 83],{'media': array([{'additional_media_info': Non...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2022-04-26


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      1000 non-null   int64  
 1   created_at                 1000 non-null   object 
 2   id                         1000 non-null   int64  
 3   id_str                     1000 non-null   int64  
 4   text                       1000 non-null   object 
 5   source                     1000 non-null   object 
 6   truncated                  1000 non-null   bool   
 7   in_reply_to_status_id      73 non-null     float64
 8   in_reply_to_status_id_str  73 non-null     float64
 9   in_reply_to_user_id        75 non-null     float64
 10  in_reply_to_user_id_str    75 non-null     float64
 11  in_reply_to_screen_name    75 non-null     object 
 12  user                       1000 non-null   object 
 13  geo                        0 non-null      float6

In [6]:
columns_list = list(df.columns)
print(columns_list)

['index', 'created_at', 'id', 'id_str', 'text', 'source', 'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms', 'display_text_range', 'extended_entities', 'possibly_sensitive', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'quoted_status_permalink', 'withheld_in_countries', 'extended_tweet', 'created_day']


In [7]:
# fetch few random instances of data from a specific column
random_instances = df['text'].sample(n=5)
random_instances

419    RT @ImranRiazKhan: عمران خان کی تقریر کا پاکست...
52     RT @Hammad_Azhar: False and ridiculous excuses...
697    RT @MaddyWithKhan: There will be a signature c...
679    RT @QasimKhanSuri: بھکاری پاکستان میں: عمران خ...
730    RT @arsched: میں پاکستان کا شہری پاکستان کا ما...
Name: text, dtype: object

In [8]:
pd.set_option('display.width', 2000)
print(df['text'].iloc[39])

RT @mshaanshahid: Rqst frm a 🇵🇰:to finance min..kindly share details of the new negotiations done with the IMF ,the people of Pakistan are…


In [9]:
print(df['text'].iloc[49])

@EjazChaudhary @usmansaeedbasra @PTIofficial #ECP_Disqualify_Lotas #MarchAgainstImportedGovt #امپورٹڈ_حکومت_نامنظور… https://t.co/cgfM4a6jE1


PREPROCESSING

In [10]:
#function for removing not needed data

import re

#def remove_urls(text, replacement_text=" "):
#    # Define a regex pattern to match URLs
#    url_pattern = re.compile(r'https?://[^\s]+|www\.[^\s]+')

#    text_without_urls = url_pattern.sub(replacement_text, text)

#    return text_without_urls

#def remove_punctuation(text):
#    #list of characters to be removed
#    characters = ['?',',','.','!']
#    for i in characters:
#        text_without_punctuation =  text_without_punctuation.replace(i,"")

#    return text_without_punctuation

def preprocess_text(text):
    # Remove all special characters, excluding Urdu characters
    text = re.sub(r'[^a-zA-Z0-9\s\u0600-\u06FF]', '', text)

    # Remove "RT"
    text = re.sub(r'RT', '', text)

    return text.strip()


In [11]:
#storing first 5 records in variable
result = df['text'].head(15).tolist()
result

['RT @fawadchaudhry: آج الیکشن کمیشن کے دفاتر کے باہر پورے ملک میں صرف ٹوکن احتجاج ہے اور بیوقوف حکومت پورا ملک ہی بند کر کے بیٹھ گئ ہے، اسلا…',
 'RT @ImranRiazKhan: عمران خان کی تقریر کا پاکستان کے 90 فیصد میڈیا میں بلیک آؤٹ',
 'RT @InsafPK: پاکستان تحریک انصاف پنجاب کا صوبائی صدر شفقت محمود اور انفارمیشن سیکرٹری مسرت جمشید چیمہ کی قیادت میں لاہور میں الیکشن کمیشن ک…',
 'RT @ZBasharit: پاکستان رمضان کے مہینے میں آزاد ہُوا تھا اور رمضان کے مہینے میں ہی ہمیں پتہ چلا کہ نہیں ابھی آزاد نہیں ہُوا \nتو مہربانی کر ک…',
 'RT @ImranRiazKhan: مبینہ طور پر ٹویٹر کی تاریخ کا سب سے بڑا ٹرینڈ #امپورٹڈ_حکومت_نامنظور بن گیا ہے۔ ہن آرام اے',
 '@CMShehbaz #امپورٹڈ_حکومت_نامنظور        #غلطی_سدھارو_الیکشن_کراؤ',
 '@ImranRiazKhan or kar b kaya sakta \n#امپورٹڈ_حکومت_نامنظور\n#MarchAgainstImportedGovt',
 'RT @PTIofficial: ہم اللہ کے سوا کسی کے آگے نہیں جھکیں گے اور نہ ہماری قوم اس امپورٹڈ حکومت کو قبول کرے گی۔ چیئرمین عمران خان \n #MarchAgains…',
 'RT @ShkhRasheed: #امپورٹڈ_حکومت_نامنظور https://t.co/Ro6H

In [12]:
#checking pre-processing on first 5 records
preprocessed_result = [preprocess_text(text) for text in result]

for item in preprocessed_result:
    print(item)

fawadchaudhry آج الیکشن کمیشن کے دفاتر کے باہر پورے ملک میں صرف ٹوکن احتجاج ہے اور بیوقوف حکومت پورا ملک ہی بند کر کے بیٹھ گئ ہے، اسلا
ImranRiazKhan عمران خان کی تقریر کا پاکستان کے 90 فیصد میڈیا میں بلیک آؤٹ
InsafPK پاکستان تحریک انصاف پنجاب کا صوبائی صدر شفقت محمود اور انفارمیشن سیکرٹری مسرت جمشید چیمہ کی قیادت میں لاہور میں الیکشن کمیشن ک
ZBasharit پاکستان رمضان کے مہینے میں آزاد ہُوا تھا اور رمضان کے مہینے میں ہی ہمیں پتہ چلا کہ نہیں ابھی آزاد نہیں ہُوا 
تو مہربانی کر ک
ImranRiazKhan مبینہ طور پر ٹویٹر کی تاریخ کا سب سے بڑا ٹرینڈ امپورٹڈحکومتنامنظور بن گیا ہے۔ ہن آرام اے
CMShehbaz امپورٹڈحکومتنامنظور        غلطیسدھاروالیکشنکراؤ
ImranRiazKhan or kar b kaya sakta 
امپورٹڈحکومتنامنظور
MarchAgainstImportedGovt
PTIofficial ہم اللہ کے سوا کسی کے آگے نہیں جھکیں گے اور نہ ہماری قوم اس امپورٹڈ حکومت کو قبول کرے گی۔ چیئرمین عمران خان 
 MarchAgains
ShkhRasheed امپورٹڈحکومتنامنظور httpstcoRo6H2MSngC
SHABAZGIL عمران خان نےکوئی کام کسی پزیرائی کے لئے نہیں کیا اور نہ ہی انہیں ایسی کوئی طلب ہے
ان 

In [13]:
#applying the pre-process function on the entire column
df['text'] = df['text'].apply(preprocess_text)

In [14]:
random_instances = df['text'].sample(n=5)
random_instances

706    httpstcorARLzAOzT9\nAnother video KarachiBlast...
647    HaqeeqatTV بھکاریوں پرانا پاکستان مبارک\n\nڈیز...
999    ImranRiazKhan مبینہ طور پر ٹویٹر کی تاریخ کا س...
58                  22ndPM PTI والے نام مینشن کریں شکریہ
914    HSodhar70 Fake PTI  Bot                       ...
Name: text, dtype: object

TRANSLATION

In [15]:
pip install deep-translator

In [16]:
#testing googletranslator library
from deep_translator import GoogleTranslator

lator = GoogleTranslator()

res1 = lator.translate('یہ ایک گاڑی ہے۔')
res2 = lator.translate('Type in English - اردو میں ٹائپ کریں')

print(res1)

This is a vehicle.


In [17]:
#testing on only first 5 PRE-PROCESSED records
from deep_translator import GoogleTranslator

lator = GoogleTranslator()

#translated_records = [lator.translate(record) for record in result]

#for record in translated_records:
#    print(record)

#processes them one by one
def next_record(preprocessed_result):
    for row in preprocessed_result:
        yield row

translated = []
for record in next_record(preprocessed_result):
    rec = lator.translate(record)
    translated.append(rec)
    print(rec)

fawadchaudhry Today there are only token protests in the entire country outside the offices of the Election Commission and the stupid government has shut down the entire country.
ImranRiazKhan Imran Khan's speech blacked out in 90% of Pakistan's media
InsafPK Pakistan Tehreek-e-Insaf Punjab Provincial President Shafqat Mehmood and Information Secretary Musrat Jamshed Cheema led the Election Commission in Lahore.
ZBasharit Pakistan was freed in the month of Ramadan and it was in the month of Ramadan that we found out that it is not free yet.
So please
ImranRiazKhan has reportedly become the biggest trend in the history of Twitter imported government disapproval. Oh relax
CMShehbaz Imported Govt unsanctioned error correction and election
Imran Riaz Khan or kar b kaya sakta
Imported government disapproved
MarchAgainstImportedGovt
PTIofficial We will not bow down to anyone except Allah and our nation will not accept this imported government. Chairman Imran Khan
 March Again
ShkhRasheed Imp

In [18]:
#translating the entire text column
def next_record(result):
    for row in result:
        yield row

translated = []

for record in next_record(df['text']):
    try:
        if record.isascii():  #check if text is in English or not, so it will remain the same
            translated.append(record)
        else:
            rec = lator.translate(record) #if its not english, it will be translated to english
            translated.append(rec)
        print(rec)
    except Exception as e:
        print(f"Error translating record: {e}")
        translated.append("")  # Append an empty string or a default value

df['translated_text'] = translated

fawadchaudhry Today there are only token protests in the entire country outside the offices of the Election Commission and the stupid government has shut down the entire country.
ImranRiazKhan Imran Khan's speech blacked out in 90% of Pakistan's media
InsafPK Pakistan Tehreek-e-Insaf Punjab Provincial President Shafqat Mehmood and Information Secretary Musrat Jamshed Cheema led the Election Commission in Lahore.
ZBasharit Pakistan was freed in the month of Ramadan and it was in the month of Ramadan that we found out that it is not free yet.
So please
ImranRiazKhan has reportedly become the biggest trend in the history of Twitter imported government disapproval. Oh relax
CMShehbaz Imported Govt unsanctioned error correction and election
Imran Riaz Khan or kar b kaya sakta
Imported government disapproved
MarchAgainstImportedGovt
PTIofficial We will not bow down to anyone except Allah and our nation will not accept this imported government. Chairman Imran Khan
 March Again
ShkhRasheed Imp

PRE-PROCESSING after TRANSLATION

In [22]:
pip install --user -U nltk

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [39]:
nltk.download('corpora')

[nltk_data] Error loading corpora: Package 'corpora' not found in
[nltk_data]     index


False

In [40]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob

def remove_stopwords(text):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    words = [word for word in TextBlob(text).words if word.lower() not in stopwords]
    return ' '.join(words)

def preprocess_text2(translated_text):
    # Remove links
    translated_text = re.sub(r'https?://[^\s]+(?=\s|$)', '', translated_text)
    #translated_text = re.sub(r'(https?://[^\s]+)|(www\.[^\s]+)(?=\W|$)', '', translated_text)
    #translated_text = re.sub(r'https?://[^"\s]+', '', translated_text)

    # Remove punctuation
    # text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    translated_text = re.sub(r'[^\w\s]+', '', translated_text)

    #convert all to lowercase
    df['translated_text'] = df['translated_text'].str.lower()

    # Load the stop words
    #stop_words = set(stopwords.words('english'))
    #df['translated_text'] = df['translated_text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
    #df['translated_text'] = df['translated_text'].apply(remove_stopwords)

    return translated_text.strip()

In [41]:
#storing first 5 records in variable
result2 = df['translated_text'].head(15).tolist()
result2

['fawadchaudhry today token protests entire country outside offices election commission stupid government shut entire country',
 'imranriazkhan imran khans speech blacked 90 pakistans media',
 'insafpk pakistan tehreekeinsaf punjab provincial president shafqat mehmood information secretary musrat jamshed cheema led election commission lahore',
 'zbasharit pakistan freed month ramadan month ramadan found free yet please',
 'imranriazkhan reportedly become biggest trend history twitter imported government disapproval oh relax',
 'cmshehbaz imported govt unsanctioned error correction election',
 'imran riaz khan kar b kaya sakta imported government disapproved marchagainstimportedgovt',
 'ptiofficial bow anyone except allah nation accept imported government chairman imran khan march',
 'shkhrasheed imported government unapproved httpstcoro6h2msngc',
 'shabazgil imran khan done work favor demand work pleasure allah taala ishq',
 'waqarsatti take care pictures two days later intellectuals s

In [42]:
#checking pre-processing on first 5 records
preprocessed_result2 = [preprocess_text2(text) for text in result2]

for item in preprocessed_result2:
    print(item)

fawadchaudhry today token protests entire country outside offices election commission stupid government shut entire country
imranriazkhan imran khans speech blacked 90 pakistans media
insafpk pakistan tehreekeinsaf punjab provincial president shafqat mehmood information secretary musrat jamshed cheema led election commission lahore
zbasharit pakistan freed month ramadan month ramadan found free yet please
imranriazkhan reportedly become biggest trend history twitter imported government disapproval oh relax
cmshehbaz imported govt unsanctioned error correction election
imran riaz khan kar b kaya sakta imported government disapproved marchagainstimportedgovt
ptiofficial bow anyone except allah nation accept imported government chairman imran khan march
shkhrasheed imported government unapproved httpstcoro6h2msngc
shabazgil imran khan done work favor demand work pleasure allah taala ishq
waqarsatti take care pictures two days later intellectuals say niazi sahib went swim alone sea perform

*removing hyperlinks & stopwords not working succesfully

In [31]:
#applying the pre-process function on the entire column
df['translated_text'] = df['translated_text'].apply(preprocess_text2)

In [ ]:
#dropping all unnecessary columns
df = df.drop(columns=[col for col in df.columns if col != 'translated_text'])
df.head()